## Cleaning epoched data

In [1]:
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')
mne.set_log_level('warning')

mne.sys_info()

Platform:      Windows-10-10.0.19041-SP0
Python:        3.8.8 (default, Feb 24 2021, 15:54:32) [MSC v.1928 64 bit (AMD64)]
Executable:    F:\anaconda3\python.exe
CPU:           Intel64 Family 6 Model 60 Stepping 3, GenuineIntel: 4 cores
Memory:        15.9 GB

mne:           0.24.dev0
numpy:         1.18.5 {blas=mkl_rt, lapack=mkl_rt}
scipy:         1.6.2
matplotlib:    3.3.4 {backend=Qt5Agg}

sklearn:       0.24.1
numba:         0.53.1
nibabel:       Not found
nilearn:       Not found
dipy:          Not found
cupy:          Not found
pandas:        1.2.3
mayavi:        Not found
pyvista:       0.32.1 {pyvistaqt=0.5.0, OpenGL 4.3.0 - Build 20.19.15.4835 via Intel(R) HD Graphics 4600}
vtk:           9.0.3
PyQt5:         5.12
pooch:         v1.5.2


In [2]:
epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs_epo.fif')
epochs

Number of events,320
Events,Auditory/Left: 72Auditory/Right: 73Button: 16Smiley: 15Visual/Left: 73Visual/Right: 71
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [3]:
epochs.apply_baseline((None, 0))

Number of events,320
Events,Auditory/Left: 72Auditory/Right: 73Button: 16Smiley: 15Visual/Left: 73Visual/Right: 71
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [4]:
epochs.plot()

<MNEBrowseFigure size 800x800 with 4 Axes>

## Reject artifacts based on channel signal amplitude

In [5]:
reject_criteria = dict(mag=3000e-15,     # 3000 fT
                       grad=3000e-13,    # 3000 fT/cm
                       eeg=150e-6,       # 150 µV
                       eog=200e-6)       # 200 µV

flat_criteria = dict(mag=1e-15,          # 1 fT
                     grad=1e-13,         # 1 fT/cm
                     eeg=1e-6)           # 1 µV

In [6]:
epochs.drop_bad(reject=reject_criteria, flat=flat_criteria)

Number of events,271
Events,Auditory/Left: 57Auditory/Right: 61Button: 15Smiley: 14Visual/Left: 67Visual/Right: 57
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [7]:
epochs.plot_drop_log()

Traceback (most recent call last):
  File "F:\anaconda3\lib\site-packages\matplotlib\backends\backend_qt5.py", line 480, in _draw_idle
    self.draw()
  File "F:\anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py", line 407, in draw
    self.figure.draw(self.renderer)
  File "F:\anaconda3\lib\site-packages\matplotlib\artist.py", line 41, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "F:\anaconda3\lib\site-packages\matplotlib\figure.py", line 1863, in draw
    mimage._draw_list_compositing_images(
  File "F:\anaconda3\lib\site-packages\matplotlib\image.py", line 131, in _draw_list_compositing_images
    a.draw(renderer)
  File "F:\anaconda3\lib\site-packages\matplotlib\artist.py", line 41, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "F:\anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py", line 411, in wrapper
    return func(*inner_args, **inner_kwargs)
  File "F:\anaconda3\lib\site-packages\matplotlib\axes\_

<Figure size 640x480 with 1 Axes>

Quite a number of Epochs were dropped due to EOG artifacts.

In [8]:
epochs['Visual'].plot_image()

[<Figure size 640x480 with 3 Axes>,
 <Figure size 640x480 with 3 Axes>,
 <Figure size 640x480 with 3 Axes>]

In [9]:
epochs.plot_sensors(ch_type='eeg')

<Figure size 640x640 with 1 Axes>

In [10]:
epochs['Visual'].plot_image(picks='EEG 060')

[<Figure size 640x480 with 4 Axes>]

## SSP

### Let's see if we can retain most the epochs, but still get rid of the EOG artifact. And while we're at it, the ECG artifacts too 😉

In [11]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=0.1, h_freq=40)

ecg_projs, ecg_events = mne.preprocessing.compute_proj_ecg(raw, n_grad=1, n_mag=1, n_eeg=0,
                                                           average=True)

eog_projs, eog_events = mne.preprocessing.compute_proj_eog(raw, n_grad=1, n_mag=1,
                                                           n_eeg=1, average=True)

<ipython-input-11-0cbe2f80884e>:10: RuntimeWarning: The unit for channel(s) STI 001, STI 002, STI 003, STI 004, STI 005, STI 006, STI 014, STI 015, STI 016 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path)


In [12]:
eog_projs

[<Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | EOG-planar--0.200-0.200-PCA-01, active : False, n_channels : 203>,
 <Projection | EOG-axial--0.200-0.200-PCA-01, active : False, n_channels : 102>,
 <Projection | EOG-eeg--0.200-0.200-PCA-01, active : False, n_channels : 59>]

In [13]:
projs = eog_projs + ecg_projs
projs

[<Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | EOG-planar--0.200-0.200-PCA-01, active : False, n_channels : 203>,
 <Projection | EOG-axial--0.200-0.200-PCA-01, active : False, n_channels : 102>,
 <Projection | EOG-eeg--0.200-0.200-PCA-01, active : False, n_channels : 59>,
 <Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | ECG-planar--0.200-0.400-PCA-01, active : False, n_channels : 203>,
 <Projection | ECG-axial--0.200-0.400-PCA-01, active : False, n_channels : 102>]

In [14]:
epochs.add_proj(projs)
epochs.plot()

<MNEBrowseFigure size 800x800 with 5 Axes>

In [15]:
epochs_cleaned = epochs.copy().apply_proj()

epochs_cleaned['Visual'].plot_image()
epochs_cleaned['Visual'].plot_image(picks='EEG 060')

[<Figure size 640x480 with 4 Axes>]

## ICA

First, start with the **raw data** again and apply a 1.0 Hz high-pass filter, which is advantegeous for ICA performance.

In [16]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=1, h_freq=40)  # High-pass with 1. Hz cut-off is recommended for ICA

<ipython-input-16-d7007c9a0285>:10: RuntimeWarning: The unit for channel(s) STI 001, STI 002, STI 003, STI 004, STI 005, STI 006, STI 014, STI 015, STI 016 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path)


<Raw | sub-01_ses-01_task-audiovisual_run-01_meg.fif, 376 x 166800 (277.7 s), ~479.0 MB, data loaded>

Read our epochs and extract which ones that were kept (**all** in our case, because we didn't apply any rejection procedure before saving the epochs in notebook 3; but this could be different in a real-world scenario, and you want to calculate ICA on the same set of epochs you're actually feeding into your analysis!)

In [23]:
epochs = mne.read_epochs(pathlib.Path('out_data') / 'epochs_epo.fif')
epochs_selection = epochs.selection
epochs_selection

320

Only keep the subset of events that corresponds to the retained epochs.

In [24]:
events, event_id = mne.events_from_annotations(raw)
events = events[epochs_selection]

Create epochs for ICA. All parameters should match **exactly** the ones of the epochs we intend to clean.

In [19]:
tmin = -0.3
tmax = 0.5
baseline = (None, 0)

epochs_ica = mne.Epochs(raw,
                        events=events,
                        event_id=event_id,
                        tmin=tmin,
                        tmax=tmax,
                        baseline=baseline,
                        preload=True)

### Finally, fit ICA!

In [20]:
epochs_ica.info

<Info | 22 non-empty values
 acq_pars: ACQch001 110113 ACQch002 110112 ACQch003 110111 ACQch004 110122 ...
 bads: 2 items (MEG 2443, EEG 053)
 ch_names: MEG 0113, MEG 0112, MEG 0111, MEG 0122, MEG 0123, MEG 0121, MEG ...
 chs: 204 Gradiometers, 102 Magnetometers, 9 Stimulus, 60 EEG, 1 EOG
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 dev_head_t: MEG device -> head transform
 dig: 146 items (3 Cardinal, 4 HPI, 61 EEG, 78 Extra)
 events: 1 item (list)
 experimenter: mne_anonymize
 file_id: 4 items (dict)
 highpass: 1.0 Hz
 hpi_results: 1 item (list)
 line_freq: 60
 lowpass: 40.0 Hz
 meas_date: 2002-12-03 19:01:10 UTC
 meas_id: 4 items (dict)
 nchan: 376
 proj_id: 1 item (ndarray)
 proj_name: mne_anonymize
 projs: PCA-v1: on, PCA-v2: on, PCA-v3: on
 sfreq: 600.6 Hz
 subject_info: 4 items (dict)
>

In [21]:
n_components = 0.8  # Should normally be higher, like 0.999!!
method = 'picard'
max_iter = 100  # Should normally be higher, like 500 or even 1000!!
fit_params = dict(fastica_it=5)
random_state = 42

ica = mne.preprocessing.ICA(n_components=n_components,
                            #max_pca_components=300,
                            method=method,
                            max_iter=max_iter,
                            fit_params=fit_params,
                            random_state=random_state)
ica.fit(epochs_ica)

<ipython-input-21-f3dc9862ec1a>:13: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs_ica)
F:\anaconda3\lib\site-packages\picard\solver.py:213: UserWarning: Picard did not converge. Final gradient norm : 4.683e-06. Requested tolerance : 1e-07. Consider increasing the number of iterations or the tolerance.
  warnings.warn('Picard did not converge. Final gradient norm : %.4g.'


Method,picard
Fit,100 iterations on epochs (153920 samples)
ICA components,28
Explained variance,80.1 %
Available PCA components,364
Channel types,"mag, grad, eeg"
ICA components marked for exclusion,—


In [25]:
ica.plot_components(inst=epochs)

[<MNEFigure size 975x781 with 20 Axes>, <MNEFigure size 975x496 with 8 Axes>]

### Detect ECG and EOG patterns

In [32]:
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw, reject=None,
                                                 baseline=(None, -0.2),
                                                 tmin=-0.5, tmax=0.5)
ecg_evoked = ecg_epochs.average()
ecg_inds, ecg_scores = ica.find_bads_ecg(
    ecg_epochs, method='ctps')


eog_epochs = mne.preprocessing.create_eog_epochs(raw, reject=None,
                                                 baseline=(None, -0.2),
                                                 tmin=-0.5, tmax=0.5)
eog_evoked = eog_epochs.average()
eog_inds, eog_scores = ica.find_bads_eog(
    eog_epochs)

components_to_exclude = ecg_inds + eog_inds
ica.exclude = components_to_exclude


[0, 14, 1]

## Plot automated artifact detection scores

In [33]:
ica.plot_scores(ecg_scores)

<Figure size 640x270 with 1 Axes>

## Plot ICA sources

In [30]:
ica.plot_sources(ecg_evoked)

<Figure size 640x480 with 1 Axes>

## Plot overlay of original and cleaned data

In [31]:
ica.plot_overlay(ecg_evoked)

<ipython-input-31-4cf3391a4078>:1: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.
  ica.plot_overlay(ecg_evoked)


<Figure size 640x480 with 3 Axes>

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Visualize artifact detection scores, ICA sources, and an overlay of original and cleaned data based on the EOG epochs.</li>
         <li>Visualize artifact detection scores, ICA sources, and an overlay of original and cleaned data based on the epochs we actually intend to analyze.</li>
    </ul>
</div>

In [34]:
epochs_cleaned = ica.apply(epochs.copy())

<ipython-input-34-fa7578ef095d>:1: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.
  epochs_cleaned = ica.apply(epochs.copy())


In [36]:
epochs_cleaned.plot(title='After ICA')
epochs.plot(title='before ICA')

<MNEBrowseFigure size 800x800 with 4 Axes>